In [ ]:
import rospy
from geometry_msgs.msg import Pose, Point
import assignment_2
import assignment_2_2022
import assignment_2_2022.msg
from assignment_2.srv import Goal
from assignment_2.msg import PlanningAction
from assignment_2.msg import PlanningActionGoal
import actionlib
import actionlib.msg
%matplotlib widget
from nav_msgs.msg import Odometry
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation

from std_srvs.srv import *
import tf
from tf.transformations import quaternion_matrix
from IPython.display import display
from sensor_msgs.msg import LaserScan
import jupyros as jr
import time
import math

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
def print_menu():
    print("Welcome to this program! Running the cell below you can do the following things:")
    print("-  Set a new goal")
    print("-  Cancel the current goal")
    print("-  Print the number of goals reached and cancelled")
    print("-  Exit from the program")

In [ ]:
posx = 0
posy = 0
goalx = 0
goaly = 0
def pos_callback(info):
    posx = info.pose.pose.position.x
    posy = info.pose.pose.position.y

In [ ]:
rospy.init_node('action_client')
client = actionlib.SimpleActionClient('/reaching_goal', assignment_2.msg.PlanningAction)
client.wait_for_server()
print("UNO")
rospy.wait_for_service('/result')
print("DUE")
serv = rospy.ServiceProxy('/result',Goal)
rospy.Subscriber("odom", Odometry, pos_callback)
    
current_state = client.get_state()
    
print_menu()
    
#Bunch of buttons to provide the user all the functionalities:
send_button = widgets.Button(description="Set a New Goal")
output1 = widgets.Output()
display(send_button, output1)

#Textareas for writing goal position:
x_text = widgets.FloatSlider(value=0,min=-8,max=8,step=.1)
output_x = widgets.Output()
y_text = widgets.FloatSlider(value=0,min=-8,max=8,step=.1)
output_y = widgets.Output()
display(x_text,output_x)
display(y_text,output_y)

#Other buttons...
cancel_button = widgets.Button(description="Cancel the Current Goal")
output2 = widgets.Output()
display(cancel_button, output2)
print_button = widgets.Button(description="Print the number of goals reached and cancelled")
output3 = widgets.Output()
display(print_button, output3)
exit_button = widgets.Button(description="Exit from the program")
output4 = widgets.Output()
display(exit_button, output4)
    
def on_cancel_clicked(cancel_button):
    with output2:
        cancel_goal()
        
def on_print_clicked(print_button):
    with output3:
        print_goal()
        
def on_exit_clicked(exit_button):
    with output4:
        exit_program()

def cancel_goal():
    
    current_state = client.get_state()
    if current_state == actionlib.GoalStatus.ACTIVE:
        client.cancel_goal()
        print("Goal Successfully Cancelled!")
        rise_cancel = True
    else:
        print("No goal is currently set!")

def print_goal():
    resp = serv()
    n_reached = resp.reach
    n_cancelled = resp.canc
    print("Number of goal reached: ")
    print(n_reached)
    print("Number of goal cancelled: ")
    print(n_cancelled)
    
def exit_program():
    send_button.close()
    cancel_button.close()
    print_button.close()
    exit_button.close()
    x_text.close()
    y_text.close()
    print("Bye Bye!")
    exit(0)
    
def on_send_clicked(send_button):
    global x_val
    global y_val
    with output1:
        
        x = x_text.value
        y = y_text.value
        
        send_goal(x,y)
        
def send_goal(x_pos,y_pos):
    
    global goalx, goaly
    
    current_state = client.get_state()
    if current_state == actionlib.GoalStatus.ACTIVE:
        print("A goal has already been set, if you want to set a new goal now, please cancel the current one")
    else:
        goal = assignment_2_2022.msg.PlanningGoal()
        goal.target_pose.pose.position.x = x_pos
        goal.target_pose.pose.position.y = y_pos
        
        goalx = x_pos
        goaly = y_pos
    
        client.send_goal(goal)
        
        print("Goal successfully sent!")
      
    
    
cancel_button.on_click(on_cancel_clicked)
print_button.on_click(on_print_clicked)
send_button.on_click(on_send_clicked)
exit_button.on_click(on_exit_clicked)


In [ ]:

class Visualiser():
    def __init__(self):
        self.fig, (self.ax, self.ax2) = plt.subplots(1,2, figsize=(10,4))
        self.ln, = self.ax.plot([], [], 'ro', label='Trajectory')
        self.posx = posx
        self.posy = posy
        self.goalx = goalx
        self.goaly = goaly
        self.x_data, self.y_data = [] , []
        self.n_reach = 0
        self.n_cancel = 0
        self.check_reach = False
        self.check_cancel = False
    def plot_init(self):
        self.ax.set_xlim(-20, 20)
        self.ax.set_ylim(-20, 20)
        return self.ln
    def odom_callback(self, msg):
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)
    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        self.posx = posx
        self.posy = posy
        self.goalx = goalx
        self.goaly = goaly
        current_state = client.get_state()
        if current_state == 3 and not self.check_reach:
            self.ax.plot(self.goalx, self.goaly, marker='o', color='green', label='Target Reached')
            self.check_reach = True
            self.n_reach += 1
            self.ax2.bar(['reached', 'deleted'], [self.n_reach, self.n_cancel], color = 'b')
        elif current_state == 2 and not self.check_cancel:
            self.ax.plot(self.goalx, self.goaly, marker='o', color='blue', label='Target Cancelled')
            self.check_cancel = True
            self.n_cancel += 1
            self.ax2.bar(['reached', 'deleted'], [self.n_reach, self.n_cancel], color = 'b')
        elif current_state == 1:
            self.ax.plot(self.goalx, self.goaly, marker='o', color='orange', label='Current Target')
            print(goalx)
        if not(current_state == 3) and self.check_reach:
            self.check_reach = False
        if not(current_state == 2) and self.check_cancel:
            self.check_cancel = False
        self.ax2.bar(['reached', 'deleted'], [self.n_reach, self.n_cancel], color = 'b')
        return self.ln

In [ ]:
global posx, posy, goalx, goaly

vis = Visualiser()
vis.ax.set_title('Plot of robot trajectory')
vis.ax.set_xlabel('Position on x')
vis.ax.set_ylabel('Position on y')
vis.ax.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.tight_layout()
vis.ax.grid()

vis.ax2.set_title('Number of goal reached and deleted')
vis.ax2.set_xlabel('Goals')
vis.ax2.set_ylabel('Number of goal')

sub_pos = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
time.sleep(0.2)
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)

plt.show(block=True)

distance_text = widgets.FloatText(description='Obstacle distance: ', disabled=False)
output_distance = widgets.Output()
display(distance_text, output_distance)

In [ ]:
def scan_callback(scan):
    min_range = scan.range_max
    for range_val in scan.ranges:
        if range_val < min_range:
            min_range = range_val
    distance_text.value=round(min_range, 3)

jr.subscribe('/scan', LaserScan, scan_callback)